# **基礎的なコードの紹介**

***必要ライブラリの準備***

In [1]:
# 必要なライブラリをインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Googleドライブをマウント
from google.colab import drive

**データの読み込み（google colabを想定）**

In [2]:
# Googleドライブをマウント
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# csvファイルを読み込み、データフレームに格納
df = pd.read_csv("/content/drive/MyDrive/カスタムデータセット/synthetic_data.csv")

In [4]:
df.info()

# データフレーム（表形式のデータ）に関する基本的な情報を表示するためのコードです。
# ここで使われている df はデータフレームの名前です。
# このコードを使うと、データの概要がわかり、どんなデータを扱っているのかを把握するのに役立ちます。

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   能力評価      4000 non-null   object 
 1   性格診断結果    5000 non-null   object 
 2   思索型:内閉性   4000 non-null   float64
 3   自制型:弱気さ   4000 non-null   float64
 4   従順性       4000 non-null   float64
 5   ﾓﾗﾄﾘｱﾑ傾向  4000 non-null   float64
 6   自己信頼性     4000 non-null   float64
 7   努力型:持続性   4000 non-null   float64
 8   積極性       4000 non-null   float64
 9   協調性       4000 non-null   float64
 10  共感性       4000 non-null   float64
 11  努力型:規則性   4000 non-null   float64
dtypes: float64(10), object(2)
memory usage: 468.9+ KB


**以上の結果から以下のようなことがわかります**

**1.データフレームの規模**
行数: データフレームには合計 5,000 行のデータが含まれています（インデックス 0 から 4,999）。
列数: 合計で 12 列のデータが含まれています。

**2.各列の情報**
列名とそのデータ型が表示されています。
データ型 object は文字列を、float64 は浮動小数点数（小数を含む数値データ）を表しています。

**3.欠損値の有無**
CUBIC総合評価 列には欠損値がなく、すべての行にデータが存在しています（5,000行すべてが non-null）。
それ以外の列（2022.9期能力評価 や各数値列）は 4,000 行しかデータがないため、1,000 行に欠損値があります。これは、データの一部が欠損していることを示しています。


---



**機械学習モデルを構築する際には、すべてのデータが数値である必要があります。**
*現在のデータフレームには、”能力評価” と ”性格診断結果” という2つの文字列（object型）のカラムがあります。
これらは数値に変換する必要があります。たとえば、ラベルエンコーディングやワンホットエンコーディングを使用してカテゴリデータを数値に変換することが考えられます。*

**また、いくつかのカラムには欠損値が含まれています**
*機械学習モデルは欠損値を扱うことができないため、これらの欠損値を埋める必要があります。
欠損値の処理方法としては、平均値や中央値で埋める、前後の値で補完する、もしくは欠損値のある行を削除するなどの方法があります。*


---



# **文字列（object型）のカラムの数値変換**

**ラベルエンコーディング**

*ラベルエンコーディングは、カテゴリ（文字データ）を数値に置き換える方法です。カテゴリの数に応じて、0から始まる整数を割り当てます。*


例えば、果物のデータがあったとします。カテゴリ: [リンゴ, バナナ, オレンジ, リンゴ, バナナ]

これをラベルエンコーディングすると、以下のようになります。

リンゴ -> 0

バナナ -> 1

オレンジ -> 2

エンコード後のデータはこうなります：[0, 1, 2, 0, 1]

**利点:**
簡単にカテゴリを数値に変換できる。

**注意点:** 数値に順序や大きさの意味がない場合でも、あたかも順序があるように見えることがあります。たとえば、0 < 1 < 2 のように数値の大小関係があると誤解されることがあります。


---



**ワンホットエンコーディング**

*ワンホットエンコーディングは、各カテゴリを「0」と「1」で表すベクトル（数値の列）に変換する方法です。各カテゴリに専用の列を作り、その列には該当するカテゴリにだけ「1」を、他はすべて「0」を入れます。*


例えば、果物のデータがあったとします。カテゴリ: [リンゴ, バナナ, オレンジ, リンゴ, バナナ]

ワンホットエンコーディングすると、次のような表になります。

```
リンゴ  バナナ  オレンジ
 1     0       0
 0     1       0
 0     0       1
 1     0       0
 0     1       0
```


**利点:**
数値に大小関係がないため、カテゴリ間に順序があるようには見えません。

**注意点:** カテゴリの数が多くなると、列の数が増えてデータが大きくなりすぎることがあります。




---


---


**今回は以下の辞書を用いて、ラベルエンコーディングを実施します**


```
# 能力評価の変換辞書
evaluation_dict = {
    'S': 5,
    'A': 4,
    'B': 3,
    'C': 2,
    'D': 1,
    '－':0
}

# 性格診断結果の変換辞書
personality_dict = {
    'S': 6,
    'A': 5,
    'B': 4,
    'B-': 3,
    'C': 2,
    'D':1,
    '－': 0
}
```



In [5]:
# 能力評価の変換辞書
evaluation_dict = {
    'S': 5,
    'A': 4,
    'B': 3,
    'C': 2,
    'D': 1,
    '－':0
}

# 性格診断結果の変換辞書
personality_dict = {
    'S': 6,
    'A': 5,
    'B': 4,
    'B-': 3,
    'C': 2,
    'D':1,
    '－': 0
}

In [6]:
# '能力評価' 列を evaluation_dict を用いて変換
df['能力評価'] = df['能力評価'].map(evaluation_dict)

# '性格診断結果' 列を personality_dict を用いて変換
df['性格診断結果'] = df['性格診断結果'].map(personality_dict)

# **欠損値の処理**

**欠損値の処理方法としては、平均値や中央値で埋める、前後の値で補完する、もしくは欠損値のある行を削除するなどの方法があります。**



```
# 平均値で欠損値を埋める
df['列名'] = df['列名'].fillna(df['列名'].mean())
```
```
# 中央値で欠損値を埋める
df['列名'] = df['列名'].fillna(df['列名'].median())
```
```
# 前の値で欠損値を埋める
df['列名'] = df['列名'].fillna(method='ffill')
```
```
# 後の値で欠損値を埋める
df['列名'] = df['列名'].fillna(method='bfill')
```
```
# データフレーム内の欠損値がある行をすべて削除
df = df.dropna()

# 特定のカラムに欠損値がある行だけを削除（例：'能力評価' カラムに欠損値がある行を削除）
df = df.dropna(subset=['能力評価'])

# 複数のカラムに欠損値がある行を削除（例：'能力評価' と '性格診断結果' カラムに欠損値がある行を削除）
df = df.dropna(subset=['能力評価', '性格診断結果'])
```





---








**今回は欠損値のある行を平均値で補完します**

In [7]:
numeric_columns = ['能力評価', '性格診断結果', '思索型:内閉性', '自制型:弱気さ', '従順性', 'ﾓﾗﾄﾘｱﾑ傾向', '自己信頼性',
                   '努力型:持続性', '積極性', '協調性', '共感性', '努力型:規則性']
df[numeric_columns] = df[numeric_columns].apply(lambda col: col.fillna(col.mean()))

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   能力評価      5000 non-null   float64
 1   性格診断結果    5000 non-null   int64  
 2   思索型:内閉性   5000 non-null   float64
 3   自制型:弱気さ   5000 non-null   float64
 4   従順性       5000 non-null   float64
 5   ﾓﾗﾄﾘｱﾑ傾向  5000 non-null   float64
 6   自己信頼性     5000 non-null   float64
 7   努力型:持続性   5000 non-null   float64
 8   積極性       5000 non-null   float64
 9   協調性       5000 non-null   float64
 10  共感性       5000 non-null   float64
 11  努力型:規則性   5000 non-null   float64
dtypes: float64(11), int64(1)
memory usage: 468.9 KB


**df.info()より、欠損値がすべて埋まったことが確認できます**

# **成形後のデータの保存**

In [9]:
df.to_csv('data.csv', index=False)

**dataという名前で、csv形式で保存します。**

**保存したデータを他のプログラムで使うことができます。**